# OpenAI Function Calling in LangChain

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic: Data validation library for Python

In [3]:
# Normal class
class User:
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age

foo = User(name="Joe", age="x") # works
print("Object:", foo)
print("Name:", foo.name)
print("Age:", foo.age)

Object: <__main__.User object at 0x0000015FE07F92B0>
Name: Joe
Age: x


In [4]:
class pUser(BaseModel):
    name: str
    age: int

# foo = pUser(name="Joe", age="x") # error
foo = pUser(name="Joe", age=32) # works
foo # pretty print

pUser(name='Joe', age=32)

In [5]:
# List of objects
class pClass(BaseModel):
    students: List[pUser]

obj = pClass(
    students=[
        foo,
        pUser(name="Jane", age=35),
    ]
)
obj

pClass(students=[pUser(name='Joe', age=32), pUser(name='Jane', age=35)])

## Use Pydantic to define OpenAI fucntions

In [6]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at the airport""" # function description, required
    airport_code: str = Field(description="airport code to get the weater for") # function params, the desc is not required

In [7]:
from langchain_core.utils.function_calling import convert_to_openai_function

weather_function = convert_to_openai_function(WeatherSearch)
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at the airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get the weater for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI().bind(functions=[weather_function], function_call={"name": "WeatherSearch"})
print(model.invoke("What is the weather in CDG today?"))
print(model.invoke("Hi"))

content='' additional_kwargs={'function_call': {'arguments': '{"airport_code":"CDG"}', 'name': 'WeatherSearch'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 83, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-54964758-eda4-4615-a896-aa7c82d4f7f3-0' usage_metadata={'input_tokens': 83, 'output_tokens': 7, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='' additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 75, 'total_tokens':

## Pass multiple functions

In [9]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

artist_search = convert_to_openai_function(ArtistSearch)

In [10]:
functions = [weather_function, artist_search]
model = ChatOpenAI().bind(functions=functions)

In [11]:
print(model.invoke("What's the weather in Orly?"))
print(model.invoke("What are five songs by Dinos?"))
print(model.invoke("Hi!"))

content='' additional_kwargs={'function_call': {'arguments': '{"airport_code":"ORY"}', 'name': 'WeatherSearch'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 119, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None} id='run-52c72ba2-8d3b-4fd3-8898-9779c1f04f6f-0' usage_metadata={'input_tokens': 119, 'output_tokens': 16, 'total_tokens': 135, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='' additional_kwargs={'function_call': {'arguments': '{"artist_name":"Dinos","n":5}', 'name': 'ArtistSearch'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens